In [ ]:
import hashlib
import requests

import json
import os
 
# Refresh the MD5s in the ota_list.json file

with open('ota_list.json', 'r') as f:
    ota_list = json.load(f)

for device in ota_list.keys():

    for f in ota_list[device]['ota_files']:
        url = f['url']
        dest = f['destination']
        file = requests.get(url).content
        md5 = hashlib.md5(file)
        f['md5'] = md5.hexdigest()

with open('ota_list.json', 'w') as f:
    json.dump(ota_list, f, indent=4)

        


In [ ]:
a = ota_list['ota_files']
type(a)
a.index(a[0])



In [ ]:
# Example of how the microcontroller would check for updates

update_indices = []
device = "2b41"
localpath = "/Users/calum/git"

with open('ota_list.json', 'r') as f:
    ota_list = json.load(f)[device]['ota_files']

# Check MD5s of existing files
for f in ota_list:
    index = ota_list.index(f)
    md5_ref = f['md5']
    dest = localpath+f['destination']
    try:
        with open(dest, 'rb') as f:
            file = f.read()
        md5 = hashlib.md5(file)
        if (md5_ref != md5.hexdigest()):
            print(f"MD5 mismatch for {dest}, adding to update list")
            update_indices.append(index)
    except FileNotFoundError:
        print(f"File {dest} not found, adding to update list")
        update_indices.append(index)

if update_indices == []:
    print("No files need updating")
else:
    print("Files need updating")
    print(update_indices)

# Download the files to temporary locations
for i in update_indices:
    f = ota_list[i]
    url = f['url']
    dest = localpath+f['destination']
    dest_temp = dest+'.ota_temp'
    file = requests.get(url).content
    os.makedirs(os.path.dirname(dest_temp), exist_ok=True)
    with open(dest_temp, 'wb') as f:
        f.write(file)

# Confirm MD5s match
for i in update_indices:
    f = ota_list[i]
    md5_ref = f['md5']
    dest = localpath+f['destination']
    dest_temp = dest+'.ota_temp'
    with open(dest_temp, 'rb') as f:
        file = f.read()
    md5 = hashlib.md5(file)
    if (md5_ref == md5.hexdigest()):
        print(f"MD5 match for {dest}")
    else: 
        print(f"MD5 mismatch for {dest}")
        raise Exception(f"MD5 mismatch for {dest}")
    # Log exception and reboot normally with original files

# Overwrite the original files with the OTA temp files
for i in update_indices:
    f = ota_list[i]
    dest = localpath+f['destination']
    dest_temp = dest+'.ota_temp'
    with open(dest_temp, 'rb') as f:
        file = f.read()
    with open(dest, 'wb') as f:
        f.write(file)
        print(f"updated {dest}")
    os.remove(dest_temp)
